In [1]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

In [21]:
import psycopg2

pg_host = "localhost"
pg_database = "graph"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [22]:
# Example: Select data
select_query = "SELECT distinct(source_id) FROM result_llm WHERE method = 'EUCLIDEAN'"
pg_cursor.execute(select_query)
results = pg_cursor.fetchall()
source_llm = []
for row in results:
    source_llm.append(row[0])

In [23]:
for i in range(len(source_llm)):
    records, summary, key  = driver.execute_query("""
    MATCH (req:Requirement) WHERE req.id = $id
    WITH req
    MATCH (target:Requirement) WHERE target.id <> $id
    WITH target, vector.similarity.euclidean(target.sentence_embedding, req.sentence_embedding) as score
    RETURN target.id as id, score ORDER BY score DESC LIMIT 5""", id=source_llm[i])
    insert_query = """
        INSERT INTO result_option_llm (source_id, option_id, similarity, method) 
        VALUES (%s, %s, %s, %s)
    """
    for record in records:
        pg_cursor.execute(insert_query, (source_llm[i], record['id'], record['score'], 'EUCLIDEAN'))
        pg_conn.commit()

In [14]:
# Example: Select data
select_query = "SELECT distinct(source_id) FROM result_graph WHERE method = 'EUCLIDEAN'"
pg_cursor.execute(select_query)
results = pg_cursor.fetchall()
source_graph = []
for row in results:
    source_graph.append(row[0])

In [15]:
for i in range(len(source_graph)):
    records, summary, key  = driver.execute_query("""
    MATCH (req:Requirement) WHERE req.id = $id
    WITH req
    MATCH (target:Requirement) WHERE target.id <> $id
    WITH target, vector.similarity.cosine(target.`fastrp-embedding`, req.`fastrp-embedding`) as score
    RETURN target.id as id, score ORDER BY score DESC LIMIT 5""", id=source_graph[i])
    insert_query = """
        INSERT INTO result_option_graph (source_id, option_id, similarity, method) 
        VALUES (%s, %s, %s, %s)
    """
    for record in records:
        pg_cursor.execute(insert_query, (source_graph[i], record['id'], record['score'], 'EUCLIDEAN'))
        pg_conn.commit()